In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hailpy-m.us-central1-c.c.cncd-cncd.internal:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.57-582b2e31b8bd
LOGGING: writing to /hail-20201005-1110-0.2.57-582b2e31b8bd.log


In [2]:
for chrNum in [21]: #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']:
    
    print("Reading Mt Chr", chrNum)
    data = hl.read_matrix_table('gs://pgr-wes-30k-mt/QCed_Data/Promis30_chr' + str(chrNum) + '_QC.mt')  
    data.rows().show()
    
    
    # filter for high confiedence variants that have at least one hom-alt call
    lofHCHom = data.filter_rows((data.info.nhomvar > 0) & (data.info.lofteelof == "HC"))
    lofHCHom.rows().count()
    
    # Count the per gene hom-alt LoF cases 
    t = lofHCHom.group_rows_by(lofHCHom.info.gene)
    t = t.aggregate(homCount = hl.agg.sum(lofHCHom.info.nhomvar))
    mt_grouped = lofHCHom.group_rows_by(lofHCHom.info.gene).aggregate_rows(consequences = hl.agg.collect_as_set(lofHCHom.info.nhomvar)).result()
#     aggregate(lof_hom = hl.agg.sum(mt.info.nhomvar))
    dataset_result =  lofHCHom.group_rows_by(lofHCHom.info.gene).aggregate(n_gom = hl.agg.count(lofHCHom.info.nhomvar>0))
    dataset_result = dataset_result.cache()
    dataset_result.rows().show()
    
    import hail.expr.aggregators as agg
    entries = lofHCmt.entries()
    results = (entries.group_by(gene = entries.info.gene.upper())
          .aggregate(n_ref = agg.count_where(entries.GT.is_hom_ref()), 
                     n_het = agg.count_where(entries.GT.is_het()),
                     n_hom = agg.count_where(entries.GT.is_hom_var()),
                    ))
    
    results = results.cache()

    res_hom = results.filter((results.n_hom>0))
    res_ann = res_hom.annotate(MAF_KO = res_hom.n_hom/(res_hom.n_hom+res_hom.n_het+res_hom.n_ref),
                               MAF_LoF = (2*res_hom.n_hom+res_hom.n_het)/(2*(res_hom.n_hom+res_hom.n_het+res_hom.n_ref)),
                               )
    res_ann.show()
    
    # Filter a all high-confidence LoF alleles
   
    lofHCHom = res_ann.filter_rows((res_ann.info.nhomvar > 0) & (res_ann.info.lofteelof == "HC") & (res_ann.variant_qc.AF[1] <= 0.05)).cache()
    lofHCHom.rows().count()

    import hail.expr.aggregators as agg

    entries = lofHCHom.entries()
    results = (entries.group_by(gene = entries.info.gene.upper())
          .aggregate(n_ref = agg.count_where(entries.GT.is_hom_ref()), 
                     n_het = agg.count_where(entries.GT.is_het()),
                     n_hom = agg.count_where(entries.GT.is_hom_var()),
                     samples = agg.collect_as_set(entries.s.lower())
                    )
              )

    results = results.cache()


    #write table
    dataPath2 = "gs://pgr-wes-30k-mt/Outputs/AnnotationPipeline/"
    results.export(dataPath2 + 'promis_KO'+str(chrNum)+'.txt')


Reading Mt Chr 21


+---------------+------------+------+----------+
| locus         | alleles    | rsid |     qual |
+---------------+------------+------+----------+
| locus<GRCh37> | array<str> | str  |  float64 |
+---------------+------------+------+----------+
| 21:9825932    | ["G","A"]  | NA   | 6.89e+02 |
| 21:9825935    | ["C","T"]  | NA   | 2.64e+03 |
| 21:9825936    | ["G","A"]  | NA   | 1.10e+03 |
| 21:9825936    | ["G","C"]  | NA   | 1.10e+03 |
| 21:9825939    | ["T","C"]  | NA   | 4.42e+05 |
| 21:9825941    | ["T","G"]  | NA   | 2.17e+02 |
| 21:9825949    | ["G","A"]  | NA   | 1.30e+02 |
| 21:9825965    | ["G","C"]  | NA   | 3.28e+03 |
| 21:9825971    | ["T","A"]  | NA   | 6.81e+03 |
| 21:9825971    | ["T","C"]  | NA   | 6.81e+03 |
+---------------+------------+------+----------+

+--------------------------------+--------------+
| filters                        | info.AC      |
+--------------------------------+--------------+
| set<str>                       | array<int32> |
+--------------------------------+--------------+
| {"VQSRTrancheSNP99.60to99.80"} | [21]         |
| {}                             | [98]         |
| {}                             | [15,37]      |
| {}                             | [15,37]      |
| {}                             | [9781]       |
| {"VQSRTrancheSNP99.60to99.80"} | [5]          |
| {"VQSRTrancheSNP99.60to99.80"} | [4]          |
| {}                             | [126]        |
| {}                             | [18,20,21,7] |
| {}                             | [18,20,21,7] |
+--------------------------------+--------------+

+---------------------------------------+---------+-------------------+
| info.AF                               | info.AN | info.BaseQRankSum |
+---------------------------------------+---------+-------------------+
| array<float64>                        |   int32 |           float64 |
+---------------------------------------+---------+-------------------+
| [1.70e-04]                            |  123758 |         -7.64e-01 |
| [7.73e-04]                            |  126822 |         -7.31e-01 |
| [1.17e-04,2.90e-04]                   |  127720 |         -8.98e-01 |
| [1.17e-04,2.90e-04]                   |  127720 |         -8.98e-01 |
| [7.70e-02]                            |  126788 |         -2.89e-01 |
| [3.84e-05]                            |  130376 |         -7.62e-01 |
| [3.02e-05]                            |  132544 |         -1.18e+00 |
| [9.56e-04]                            |  131784 |         -6.38e-01 |
| [1.38e-04,1.53e-04,1.61e-04,5.35e-05] |  130818 |         -5.37e-01 |
| [1.38e-04,1.53e-04,1.61e-04,5.35e-05] |  130818 |         -5.37e-01 |
+---------------------------------------+---------+-------------------+

+----------------------+---------+---------+---------+----------+----------+
| info.ClippingRankSum | info.DB | info.DP | info.DS | info.END |  info.FS |
+----------------------+---------+---------+---------+----------+----------+
|              float64 |    bool |   int32 |    bool |    int32 |  float64 |
+----------------------+---------+---------+---------+----------+----------+
|             6.30e-02 |   false |  517774 |   false |       NA | 7.11e+00 |
|             1.71e-01 |   false |  570625 |   false |       NA | 2.96e+00 |
|             6.30e-02 |   false |  578251 |   false |       NA | 2.96e+00 |
|             6.30e-02 |   false |  578251 |   false |       NA | 2.96e+00 |
|             0.00e+00 |   false |  603145 |   false |       NA | 1.46e+00 |
|             4.25e-01 |   false |  700166 |   false |       NA | 1.48e+01 |
|             0.00e+00 |   false |  756873 |   false |       NA | 9.06e+00 |
|             1.18e-01 |   false |  743239 |   false |       NA | 3.86e+00 |
|             2.38e-01 |   false |  703091 |   false |       NA | 1.89e+00 |
|             2.38e-01 |   false |  703091 |   false |       NA | 1.89e+00 |
+----------------------+---------+---------+---------+----------+----------+

+---------------------+-----

AttributeError: StructExpression instance has no field, method, or property 'nhomvar'
    Hint: use 'describe()' to show the names of all data fields.

In [3]:
for chrNum in [21]: #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X','Y']:
    
    print("Reading Mt Chr", chrNum)
    data = hl.read_matrix_table('gs://pgr-wes-30k-mt/QCed_Data/Promis30_chr' + str(chrNum) + '_QC.mt')  
    data.info.rows().show()

Reading Mt Chr 21


AttributeError: StructExpression instance has no field, method, or property 'rows'
    Hint: use 'describe()' to show the names of all data fields.

In [9]:
data.vep.describe()

--------------------------------------------------------
Type:
        struct {
        assembly_name: str, 
        allele_string: str, 
        ancestral: str, 
        colocated_variants: array<struct {
            aa_allele: str, 
            aa_maf: float64, 
            afr_allele: str, 
            afr_maf: float64, 
            allele_string: str, 
            amr_allele: str, 
            amr_maf: float64, 
            clin_sig: array<str>, 
            end: int32, 
            eas_allele: str, 
            eas_maf: float64, 
            ea_allele: str, 
            ea_maf: float64, 
            eur_allele: str, 
            eur_maf: float64, 
            exac_adj_allele: str, 
            exac_adj_maf: float64, 
            exac_allele: str, 
            exac_afr_allele: str, 
            exac_afr_maf: float64, 
            exac_amr_allele: str, 
            exac_amr_maf: float64, 
            exac_eas_allele: str, 
            exac_eas_maf: float64, 
            exac_fin_all